In [ ]:
# Import libraries

import numpy as np
import statistics 
import matplotlib.pyplot as plt
%matplotlib inline
from numpy.linalg import norm

from random import seed
from random import randrange
from math import sqrt
import time
from collections import Counter

In [ ]:
# Load in dataset:
data = np.loadtxt("zipcombo.dat")

# Shuffle rows of data matrix
np.random.shuffle(data)

# Slit data into 80% train set and 20% test set
test_data = data[:int(np.round(0.2*9298)),:]
train_data = data[int(np.round(0.2*9298)):,:]

## Single Decision Tree Code

In [ ]:
# Function to split and evaluate each split using Gini index
def test_split(index, dataset):
    total_left = 0 
    total_right = dataset.shape[0] 
    left_proportions = np.zeros((1,10)) 
    right_proportions = np.zeros((1,10))
    gini_index = np.inf 
    
    # sort chosen feature in order, from smallest to largest value
    indices = np.argsort(dataset[:,index]) 
    order_dataset = dataset[indices,0:index+1:index]
    # to get number of repeated feature values
    feature_values, feature_counts = np.unique(order_dataset[:,1], return_counts = True) 
   
    # place everything in the 'right' cluster first, i.e. record how many times each label occured in the right_proporions vector 
    for i in range(order_dataset.shape[0]):
        right_proportions[0,int(order_dataset[i,0])] += 1 

    row_index = 0
    
    # Go through feature column, one-by-one record corresponding label in left_proportions vector, remove from right_proportions vector 
    for feature_value, feature_count in zip(feature_values[:-1], feature_counts[:-1]):
        # to account for repeated feature values
        for j in range(feature_count):
            left_proportions[0,int(order_dataset[row_index,0])] += 1
            total_left += 1
            right_proportions[0,int(order_dataset[row_index,0])] -= 1
            total_right -= 1
            row_index += 1
        
        # to avoid dividing by 0    
        if total_left == 0 or total_right == 0:
            continue
        
        # find Gini index after each split ('movement of label' from right_proportions to left_proportions)    
        p_left = (left_proportions / total_left).dot((left_proportions / total_left).T)
        p_right = (right_proportions / total_right).dot((right_proportions / total_right).T)
        
        gini_left = (1.0 - p_left) * (total_left / dataset.shape[0])
        gini_right = (1.0 - p_right) * (total_right / dataset.shape[0])
        gini = gini_left + gini_right
        
        # record row index and index value where the split resulted in the lowest Gini index
        if gini < gini_index:
            gini_index = gini
            global best_split_value
            best_split_value = order_dataset[row_index,1]
            global best_row_index
            best_row_index = row_index
     
    return order_dataset, left_proportions, right_proportions, gini_index, index, best_split_value, best_row_index

# Function to get two split datasets at best split position (split which yeilds lowest Gini index)
def get_split(dataset, n_features):
    b_score = np.inf
    # sample features without replacement 
    features_array = np.random.choice(np.arange(1,len(dataset[0])), size = (n_features,), replace = False)
    features = features_array.tolist()
    
    for index in features:
        ordered, left_prop, right_prop, gini_index, index, best_split, best_row_index = test_split(index, dataset) 
        
        # record feature index and feature value for best split (with lowest Gini Index)
        if gini_index < b_score:
            global b_index, b_row_index, b_value
            b_index, b_value, b_score, b_row_index, b_left, b_right = index, best_split, gini_index, best_row_index, left_prop, right_prop 
    
    # Get seperate datasets 'left' and 'right' where split occurs  
    indices = np.argsort(dataset[:,b_index])
    order_dataset = dataset[indices,0:]
    left = order_dataset[:b_row_index]
    right = order_dataset[b_row_index:]
    
    return {'index':b_index, 'value':b_value, 'left_group':left, 'right_group':right} 

# Get majority vote for a group of data points (used to evaluate prediction of leaves)
def to_terminal(group):
    unique_vals = np.unique(group[:,0], return_counts = True)
    
    return unique_vals[0][np.argmax(unique_vals[1])] 

# Create function to either keep splitting sub-groups ('left' and 'right'), or evaluate prediction of sub-groups
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['left_group'], node['right_group']
    del node['left_group'] 
    del node['right_group'] 
    
    # in case no split exists
    if right.shape[0] == 0:
        node['left'] = node['right'] = to_terminal(np.vstack((left,right))) 
        return
    
    # if maximum tree depth has been reached
    if depth >= max_depth: 
        node['left'], node['right'] = to_terminal(left), to_terminal(right) 
        return
    
    # if minimum number of data points in 'left' leaf is reached evaluate, or if not split further 
    if left.shape[0] <= min_size: 
        node['left'] = to_terminal(left) 
    else:
        node['left'] = get_split(left, n_features) # if not split further
        split(node['left'], max_depth, min_size, n_features, depth+1)
    
    # if minimum number of data points in 'right' leaf is reached evaluate, or if not split further
    if right.shape[0] <= min_size: # do same for right node
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

# Function to build a single decision tree
def build_tree(dataset, max_depth, min_size, n_features):
    root = get_split(dataset, n_features)
    split(root, max_depth, min_size, n_features, 1) # greedy partition
    return root

# Function to make a prediction on a single decision tree
def predict(dataset, node, row):
    if dataset[row,[node['index']]] < node['value']:
        if isinstance(node['left'], dict):
            return predict(dataset, node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(dataset, node['right'], row)
        else:
            return node['right']

## Random Forest Code

In [ ]:
# Function to sample with replacements from original dataset to create ensemble of trees
def subsample(dataset):
    sample = np.zeros((dataset.shape[0],dataset.shape[1]))
    sample_idx = np.random.choice(np.arange(dataset.shape[0]), size = (dataset.shape[0],), replace = True)
    s_row_index = 0
    for i in sample_idx.tolist():
        sample[s_row_index] = dataset[i]
        s_row_index += 1
        
    return sample

# create ensemble of random forests:
def create_ensemble(dataset, test, max_depth, min_size, n_features, n_trees):
    predictions_test = np.zeros((1,test.shape[1]))
    predictions_train = np.zeros((1,dataset.shape[1]))
    for i in range(n_trees):
        sample = subsample(dataset)
        tree = build_tree(sample, max_depth, min_size, n_features)
        
        prediction_test = [predict(test, tree, row) for row in range(test.shape[0])]
        prediction_train = [predict(dataset, tree, row) for row in range(dataset.shape[0])]
        
        if i == 0:
            predictions_test = np.array(prediction_test)
            predictions_train = np.array(prediction_train)
        else:
            predictions_test = np.vstack((predictions_test, np.array(prediction_test)))
            predictions_train = np.vstack((predictions_train, np.array(prediction_train)))
    # Get overall prediction:
    final_predictions_test = np.zeros((1,test.shape[0]))
    final_predictions_train = np.zeros((1,dataset.shape[0]))
    # get majority vote for each prediction in ensemble (prediction on test dataset) 
    for i in range(test.shape[0]):
        all_values_test = []
        for j in range(n_trees):
            all_values_test.append(predictions_test[j,i])
        values_test = Counter(all_values_test)
        value_test, count_test = (values_test).most_common()[0]
        final_predictions_test[:,i] = value_test
        
    # get majority vote for each prediction in ensemble (prediction on train dataset) 
    for i in range(dataset.shape[0]):
        all_values_train = []
        for j in range(n_trees):
            all_values_train.append(predictions_train[j,i])
        values_train = Counter(all_values_train)
        value_train, count_train = (values_train).most_common()[0]
        final_predictions_train[:,i] = value_train
    
    return final_predictions_test, final_predictions_train 

In [ ]:
# run model (to test):
n_trees = 100
dataset = train_data

max_depth = 10
min_size = 5
n_features = int(sqrt(len(dataset[1])-1))

# Record how long model takes
start = time.time()
start_m = time.time()
final_predictions_test, final_predictions_train = create_ensemble(dataset, test_data, max_depth, min_size, n_features, n_trees)
print('time take was:', time.time() - start_m)

In [ ]:
# See if accuracy values are reasonable and match that of Sklearn
accuracy_train = (np.sum(final_predictions_train == train_data[:,0]) / train_data.shape[0]) * 100
print(accuracy_train)

accuracy_test = (np.sum(final_predictions_test == test_data[:,0]) / test_data.shape[0]) * 100
print(accuracy_test)

## Q1.1 Repeated with Random Forest

In [ ]:
# Experiment to find no. of trees to vary in cross-val (also experiment other parameters to see what variables have the biggest inpact on performance):
max_depth = 10
min_size = 1
n_features = int(sqrt(len(dataset[1])-1))
for n_trees in [3, 10, 30, 50, 70, 100]:
    print('number of trees', n_trees)
    final_predictions_test, final_predictions_train = create_ensemble(train_data, test_data, max_depth, min_size, n_features, n_trees)
    accuracy_train = np.sum(final_predictions_train == train_data[:,0]) / train_data.shape[0] * 100
    print('train accuracy:', accuracy_train)
    
    accuracy_test = np.sum(final_predictions_test == test_data[:,0]) / test_data.shape[0] * 100
    print('test accuracy:', accuracy_test)

In [ ]:
# Do 20 runs for the various hyperparameters we selected:
# Set this up but code takes too long (two days to get what I want) . Code is not set up to run multiple decision trees in parallel like the way 'Sklearn' does
# Since code too took long, I initially thought to fix the number of trees to 30 and just vary the minumim number of samples in each leaf (but this parameter was found to not make a big enough impact on the test error)
# Therefore, the Sklearn was set-up so that we could also vary the number of trees along with the minimum number of samples in good time

n_trees = 30
for min_size in [1, 3, 5, 7, 9, 11, 13]:
    print('min size:',min_size)
    
    train_total_error = []
    test_total_error = [] 
    
    for k in range(20):
        print('run:',k)
        # shuffle data before each run
        np.random.shuffle(data)

        # Slit data into 80% train set and 20% test set
        test_data = data[:int(np.round(0.2*9298)),:]
        train_data = data[int(np.round(0.2*9298)):,:]

        final_predictions_test, final_predictions_train = create_ensemble(train_data, test_data, max_depth, min_size, n_features, n_trees)
        accuracy_train = np.sum(final_predictions_train == train_data[:,0]) / train_data.shape[0] * 100
        print('train accuracy:', accuracy_train)

        accuracy_test = np.sum(final_predictions_test == test_data[:,0]) / test_data.shape[0] * 100
        print('test accuracy:', accuracy_test)

        train_total_error.append(100 - accuracy_train)
        test_total_error.append(100 - accuracy_test)

    print('min size is', min_size, ' mean of total train error ', np.mean(train_total_error) , 'standard deviation', (statistics.stdev(train_total_error)), ' and test mistakes ', np.mean(test_total_error), 'standard deviation', (statistics.stdev(test_total_error)), ) 

    